# 텐서플로우 사용하기

In [21]:
import tensorflow as tf

In [22]:
tf.VERSION

'1.14.0'

In [48]:
# 변수 : 학습되는 파라미터 (weight, bias)
w1 = tf.Variable(3.0)
w2 = tf.Variable(5.0, name = 'w2')
w3 = tf.Variable(3.0, name = 'w3')
f = w1 + w2 + w3

In [26]:
sess = tf.Session() #그래프가 정의되는 공간

In [27]:
sess.run(w1.initializer)
sess.run(w2.initializer)

In [28]:
sess.run(w1.initializer) # malloc

In [52]:
# 1. session 정의
sess = tf.Session() #그래프가 정의되는 공간
init = tf.global_variables_initializer() # 전부다 초기화
sess.run(init)
# print(w1.eval()) # session 단독으로 시용해서, eval 함수를 사용 할 수 없다
print(sess.run(w1))
print(sess.run(w2))
print(sess.run(w3))
print(sess.run(f))

3.0
5.0
3.0
11.0


In [55]:
# 2. with 사용 
with tf.Session() as sess: 
    sess.run(init)
    w1.initializer.run()
    w2.initializer.run()
    print(w1.eval()) # with 시용해서, eval 함수를 사용 할 수 있음 -> 평가하다, 값을 가져오는 용도로 사용
    print(w2.eval()) 
    print(w3.eval()) 
    print(f.eval()) 

3.0
5.0
3.0
11.0


In [46]:
# 3. with으로 만드는 세션과 동일
sess = tf.InteractiveSession()
sess.run(init)
w2.initializer.run()
print(w1.eval())
print(w2.eval())

3
5


In [56]:
w1.graph is tf.get_default_graph()

True

In [60]:
graph = tf.Graph()
with graph.as_default():
    w5 = tf.Variable(10.0)
    
w5.graph is graph

True

In [61]:
w1.graph is graph

False

In [62]:
w5 = tf.Variable(10.0)
w5

<tf.Variable 'Variable_5:0' shape=() dtype=float32_ref>

In [63]:
tf.reset_default_graph()

In [64]:
a = tf.Variable(10.0)
b = tf.Variable(1.0)

In [67]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(a.eval())
    print(b.eval())
    print(sess.run([a, b]))

10.0
1.0
[10.0, 1.0]


# 선형 회귀

In [106]:
from sklearn.datasets import california_housing
import numpy as np

In [107]:
data = california_housing

In [108]:
data.fetch_california_housing?

In [109]:
house_x, house_y = california_housing.fetch_california_housing(return_X_y=True)
house_x = np.concatenate((house_x, np.ones_like(house_x[:, :1])), -1)
house_x.shape, house_y.shape, house_x.mean(axis=0)

((20640, 9),
 (20640,),
 array([ 3.87067100e+00,  2.86394864e+01,  5.42899974e+00,  1.09667515e+00,
         1.42547674e+03,  3.07065516e+00,  3.56318614e+01, -1.19569704e+02,
         1.00000000e+00]))

In [110]:
x = tf.constant(house_x, dtype=tf.float32, name='x') #상수화
y = tf.constant(house_y.reshape([-1, 1]), dtype=tf.float32, name = 'y')

In [111]:
xt = tf.transpose(x)
xt

<tf.Tensor 'transpose_3:0' shape=(9, 20640) dtype=float32>

In [112]:
# theta
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(xt, x)), xt), y)
theta

<tf.Tensor 'MatMul_12:0' shape=(9, 1) dtype=float32>

In [113]:
sess = tf.Session()
sess.run(theta)

array([[ 4.3646225e-01],
       [ 9.3629127e-03],
       [-1.1130223e-01],
       [ 6.6694891e-01],
       [-4.3296104e-06],
       [-3.7828966e-03],
       [-4.2364365e-01],
       [-4.3689999e-01],
       [-3.7147003e+01]], dtype=float32)

# 그라디언트 자동 계산

In [132]:
x = house_x.astype(np.float32)
y = tf.constant(house_y.reshape([-1, 1]), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_normal(shape=[x.shape[-1],1]))
h = tf.matmul(x, theta)
cost = tf.reduce_mean((h-y)**2)
grad = tf.
(ys=cost, xs=theta)[0]
grad

<tf.Tensor 'gradients_9/MatMul_27_grad/MatMul_1:0' shape=(9, 1) dtype=float32>

In [1080]:
# theta = theta - learning_rate * grad
learning_rate = 0.0000001
train_op = tf.assign(theta,  theta - learning_rate*grad)
sess.run(tf.global_variables_initializer())

In [1967]:
sess.run(train_op)
sess.run([cost, grad])

[176.79208, array([[   2.3915138],
        [ 388.8872   ],
        [   8.824268 ],
        [   3.0036256],
        [ -25.78993  ],
        [ 124.42716  ],
        [ 112.93222  ],
        [-379.70163  ],
        [   3.1422892]], dtype=float32)]

파라미터 경사 = tf.gradients(코스트, 파라미터)

In [1991]:
y = [10.0]
x = [1.0]
w = tf.constant([1.]) # 1->10 w = w - learning_rate * gradient
h = w * x
cost = (h - y) ** 2
g = tf.gradients(cost, [w]) # 파라미터 경사 = tf.gradients(코스트, 파라미터)
sess.run(g)

[array([-18.], dtype=float32)]